# Dataset Documentation for Coderdata
### Table of Contents:
* Publication of Origin
* Data Modalities Available
* Data Transformations
* Samples x Drugs combinations
  

## Publication of Origin
BeatAML -  
bladderpdo
broad_sanger
cptac
crcpdo
hcmi
lincs
liverpdo
mpnst
novartispdx
pancpdo
sarcpdo

In [28]:
from pathlib import Path
from copy import deepcopy

import coderdata as cd

import matplotlib.pyplot as plt
import math
import os
import glob

import pandas as pd
import numpy as np

In [22]:
# Load in all Datasets
dataset_names = ['hcmi','beataml','mpnst','pancpdo','cptac','sarcpdo','colorectal','bladderpdo','liver','novartis']
broad_sanger_datasets = ["ccle","ctrpv2","fimm","gdscv1","gdscv2","gcsi","prism","nci60"]

### Reading in # of Samples and Drugs

In [23]:
# samples first
for dataset in dataset_names:
    sample_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_samples.*".format(dataset))
    if sample_file:
        samples_df = pd.read_csv(sample_file[0], sep="\t" if sample_file[0].endswith(".tsv") else ",")
        print(samples_df['improve_sample_id'].nunique())
    else:
        print(f"No sample file found for dataset: {dataset}")

886
1022
50
70
1139
36
61
134
62
386


In [24]:
# drugs next
for dataset in dataset_names:
    drug_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_drugs.*".format(dataset))
    if drug_file:
        drugs_df = pd.read_csv(drug_file[0], sep="\t" if drug_file[0].endswith(".tsv") else ",")
        print(drugs_df['improve_drug_id'].nunique())
    else:
        print(f"No drug file found for dataset: {dataset}")

No drug file found for dataset: hcmi
164
25
25
No drug file found for dataset: cptac
34
10
50
76
25


In [27]:
# now getting unique sample-drug pairs in the experimental data
for dataset in dataset_names:
    exp_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_experiments.*".format(dataset))
    if exp_file:
        exp_df = pd.read_csv(exp_file[0], sep="\t" if exp_file[0].endswith(".tsv") else ",")
        print(exp_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0])
    else:
        print(f"No experimental data file found for dataset: {dataset}")

No experimental data file found for dataset: hcmi
23662
212
290
No experimental data file found for dataset: cptac
276
140
3300
4453
1766


In [35]:
# now doing rnaseq x drugs
# going to need rnaseq data and experimental data
for dataset in dataset_names:
    exp_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_experiments.*".format(dataset))
    transcriptomics_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_transcriptomics.*".format(dataset))
    if transcriptomics_file:
        transcriptomics_df = pd.read_csv(transcriptomics_file[0], sep="\t" if transcriptomics_file[0].endswith(".tsv") else ",")
        all_transcriptomics_samples = transcriptomics_df['improve_sample_id'].unique() # unique samples with transcriptomics data
    if exp_file:
        exp_df = pd.read_csv(exp_file[0], sep="\t" if exp_file[0].endswith(".tsv") else ",")
        # now filter to only those samples with transcriptomics data
        exp_transcriptomics_df = exp_df[exp_df['improve_sample_id'].isin(all_transcriptomics_samples)]
        number_of_unique_sample_drug_pairs = exp_transcriptomics_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0]
        print(number_of_unique_sample_drug_pairs)
    else:
        print(f"No experimental data file found for dataset: {dataset}")

No experimental data file found for dataset: hcmi
3033
163
180
No experimental data file found for dataset: cptac
234
60
840
4453
1734


In [38]:
# rnaseq x mutations x drugs 
# loading in mutation data
for dataset in dataset_names:
    mutation_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_mutations.*".format(dataset))
    if mutation_file:
        mutations_df = pd.read_csv(mutation_file[0], sep="\t" if mutation_file[0].endswith(".tsv") else ",")
        all_mutation_samples = mutations_df['improve_sample_id'].unique() # unique samples with mutation data
    exp_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_experiments.*".format(dataset))
    transcriptomics_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_transcriptomics.*".format(dataset))
    if transcriptomics_file:
        transcriptomics_df = pd.read_csv(transcriptomics_file[0], sep="\t" if transcriptomics_file[0].endswith(".tsv") else ",")
        all_transcriptomics_samples = transcriptomics_df['improve_sample_id'].unique() # unique samples with transcriptomics data
    if exp_file:  
        exp_df = pd.read_csv(exp_file[0], sep="\t" if exp_file[0].endswith(".tsv") else ",")
        # now filter to only those samples with transcriptomics data and mutation data
        exp_transcriptomics_mutation_df = exp_df[exp_df['improve_sample_id'].isin(all_transcriptomics_samples) & exp_df['improve_sample_id'].isin(all_mutation_samples)]
        number_of_unique_sample_drug_pairs = exp_transcriptomics_mutation_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0]
        print(number_of_unique_sample_drug_pairs)
    else:
        print(f"No experimental data file found for dataset: {dataset}")

No experimental data file found for dataset: hcmi
2905
163
175
No experimental data file found for dataset: cptac
187
60
640
4453
1734


In [43]:
# adding to the dataframe
datset_stats = pd.DataFrame(columns=['dataset','num_samples','num_drugs','num_sample_drug_pairs','num_sample_drug_transcript_pairs','num_sample_drug_transcript_mutation_pairs','num_sample_drug_transcript_copynum_pairs'])
for dataset in dataset_names:
    sample_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_samples.*".format(dataset))
    drug_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_drugs.*".format(dataset))
    exp_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_experiments.*".format(dataset))
    transcriptomics_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_transcriptomics.*".format(dataset))
    mutation_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_mutations.*".format(dataset))
    copy_number_file = glob.glob("/Users/imal967/pnnl/projects/coderdata/all_datasets/{}_copy_number.*".format(dataset)) 
    # getting number of samples
    if sample_file:
        samples_df = pd.read_csv(sample_file[0], sep="\t" if sample_file[0].endswith(".tsv") else ",")
        num_samples = samples_df['improve_sample_id'].nunique() # unique samples
    else:
        print(f"Missing files for dataset: {dataset}")
        num_samples = np.nan
    # getting number of drugs
    if drug_file:
        drugs_df = pd.read_csv(drug_file[0], sep="\t" if drug_file[0].endswith(".tsv") else ",")
        num_drugs = drugs_df['improve_drug_id'].nunique() # unique drugs
    else:
        print(f"Missing files for dataset: {dataset}")
        num_drugs = np.nan
    # getting number of unique sample-drug pairs
    if exp_file:
        exp_df = pd.read_csv(exp_file[0], sep="\t" if exp_file[0].endswith(".tsv") else ",")
        num_sample_drug_pairs = exp_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0]
    else:
        num_sample_drug_pairs = np.nan
    # getting number of unique sample-drug pairs with transcriptomics data
    if transcriptomics_file and exp_file:
        transcriptomics_df = pd.read_csv(transcriptomics_file[0], sep="\t" if transcriptomics_file[0].endswith(".tsv") else ",")
        all_transcriptomics_samples = transcriptomics_df['improve_sample_id'].unique() # unique samples with transcriptomics data
        # now filter to only those samples with transcriptomics data
        exp_transcriptomics_df = exp_df[exp_df['improve_sample_id'].isin(all_transcriptomics_samples)]
        num_sample_drug_transcript_pairs = exp_transcriptomics_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0]
    else:
        num_sample_drug_transcript_pairs = np.nan
    # unique sample-drug pairs with mutation data and transcriptomics data
    if transcriptomics_file and mutation_file and exp_file:
        mutations_df = pd.read_csv(mutation_file[0], sep="\t" if mutation_file[0].endswith(".tsv") else ",")
        all_mutation_samples = mutations_df['improve_sample_id'].unique() # unique samples with mutation data
        all_transcriptomics_samples = transcriptomics_df['improve_sample_id'].unique() # unique samples with transcriptomics data
        # now filter to only those samples with transcriptomics data and mutation data
        exp_transcriptomics_mutation_df = exp_df[exp_df['improve_sample_id'].isin(all_transcriptomics_samples) & exp_df['improve_sample_id'].isin(all_mutation_samples)]
        num_sample_drug_transcript_mutation_pairs = exp_transcriptomics_mutation_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0]
    else:
        num_sample_drug_transcript_mutation_pairs = np.nan
    # unique sample-drug pairs with copynum data and transcriptomics data
    if transcriptomics_file and copy_number_file and exp_file:
        copy_number_df = pd.read_csv(copy_number_file[0], sep="\t" if copy_number_file[0].endswith(".tsv") else ",")
        all_copy_number_samples = copy_number_df['improve_sample_id'].unique() # unique samples with copy number data
        all_transcriptomics_samples = transcriptomics_df['improve_sample_id'].unique() # unique samples with transcriptomics data
        # now filter to only those samples with transcriptomics data and copy number data
        exp_transcriptomics_copy_number_df = exp_df[exp_df['improve_sample_id'].isin(all_transcriptomics_samples) & exp_df['improve_sample_id'].isin(all_copy_number_samples)]
        num_sample_drug_transcript_copynum_pairs = exp_transcriptomics_copy_number_df[['improve_sample_id','improve_drug_id']].drop_duplicates().shape[0]
    else:
        num_sample_drug_transcript_copynum_pairs = np.nan
    # adding to dataframe
    datset_stats = pd.concat([datset_stats, pd.DataFrame({'dataset':[dataset],'num_samples':[num_samples],'num_drugs':[num_drugs],'num_sample_drug_pairs':[num_sample_drug_pairs],'num_sample_drug_transcript_pairs':num_sample_drug_transcript_pairs,'num_sample_drug_transcript_mutation_pairs':num_sample_drug_transcript_mutation_pairs,'num_sample_drug_transcript_copynum_pairs':num_sample_drug_transcript_copynum_pairs})], ignore_index=True)


Missing files for dataset: hcmi
Missing files for dataset: cptac


In [44]:
datset_stats

,dataset,num_samples,num_drugs,num_sample_drug_pairs,num_sample_drug_transcript_pairs,num_sample_drug_transcript_mutation_pairs,num_sample_drug_transcript_copynum_pairs
0,hcmi,886,NaN,NaN,NaN,NaN,NaN
1,beataml,1022,164,23662,3033,2905,NaN
2,mpnst,50,25,212,163,163,163
3,pancpdo,70,25,290,180,175,175
4,cptac,1139,NaN,NaN,NaN,NaN,NaN
5,sarcpdo,36,34,276,234,187,NaN
6,colorectal,61,10,140,60,60,60
7,bladderpdo,134,50,3300,840,640,640
8,liver,62,76,4453,4453,4453,4453
9,novartis,386,25,1766,1734,1734,1723
